In [2]:
import csv
from collections import deque
import elasticsearch
from elasticsearch import helpers
import re


In [5]:
def readMovies():
    csvfile = open('ml-latest-small/movies.csv', 'r', encoding='utf-8', errors='ignore')
    
    reader = csv.DictReader( csvfile )
    for line in reader:
        movie = {}
        title = re.sub(" \(.*\)$", "", re.sub('"','', line['title']))
        year = line['title'][-5:-1]
        if (not year.isdigit()):
            year = "2016"
        genres = line['genres']
        movie['movieId'] = int(line['movieId'])
        movie['title'] = title
        movie['year'] = int(year)
        movie['genres'] = genres
        yield movie

In [6]:
es = elasticsearch.Elasticsearch(["http://89.233.105.243:9200"], http_auth=('elastic', 'changeme'))

es.indices.delete(index="movies",ignore=404)
deque(helpers.parallel_bulk(es,readMovies(),index="movies"), maxlen=0)
es.indices.refresh()

<ipython-input-6-b2779f8cd303>:1: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = elasticsearch.Elasticsearch(["http://89.233.105.243:9200"], http_auth=('elastic', 'changeme'))
<ipython-input-6-b2779f8cd303>:3: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index="movies",ignore=404)


ObjectApiResponse({'_shards': {'total': 22, 'successful': 16, 'failed': 0}})